In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## चरण 1: संरचित आउटपुटसाठी Pydantic मॉडेल्स परिभाषित करा

हे मॉडेल्स एजंट्स परत करणार्‍या **स्कीमा** परिभाषित करतात. Pydantic सह `response_format` वापरण्याचे फायदे:
- ✅ प्रकार-सुरक्षित डेटा काढणे
- ✅ स्वयंचलित सत्यापन
- ✅ मुक्त-पाठ प्रतिसादांमधून कोणतेही पार्सिंग त्रुटी नाहीत
- ✅ फील्ड्सवर आधारित सोपे सशर्त रूटिंग


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## चरण 2: हॉटेल बुकिंग साधन तयार करा

हे साधन **availability_agent** वापरून तपासेल की खोल्या उपलब्ध आहेत का. आम्ही `@ai_function` डेकोरेटर वापरतो:
- Python फंक्शनला AI-कॉल करण्यायोग्य साधनात रूपांतरित करण्यासाठी
- LLM साठी स्वयंचलित JSON स्कीमा तयार करण्यासाठी
- पॅरामीटरची पडताळणी हाताळण्यासाठी
- एजंटद्वारे स्वयंचलित कॉलिंग सक्षम करण्यासाठी

या डेमोसाठी:
- **स्टॉकहोम, सिएटल, टोकियो, लंडन, अॅमस्टरडॅम** → खोल्या उपलब्ध आहेत ✅
- **इतर सर्व शहरं** → खोल्या उपलब्ध नाहीत ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## चरण ३: रूटिंगसाठी अटींच्या फंक्शन्सची व्याख्या करा

हे फंक्शन्स एजंटच्या प्रतिसादाचे निरीक्षण करतात आणि कार्यप्रवाहात कोणता मार्ग निवडायचा ते ठरवतात.

**महत्त्वाचा नमुना:**
1. तपासा की संदेश `AgentExecutorResponse` आहे
2. संरचित आउटपुट (Pydantic मॉडेल) पार्स करा
3. रूटिंग नियंत्रित करण्यासाठी `True` किंवा `False` परत करा

कार्यप्रवाह **एजेस** वर या अटींचे मूल्यांकन करेल, पुढे कोणता एक्झिक्युटर वापरायचा ते ठरवण्यासाठी.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## चरण 4: कस्टम डिस्प्ले एक्झिक्युटर तयार करा

एक्झिक्युटर्स हे वर्कफ्लो घटक आहेत जे रूपांतरणे किंवा साइड इफेक्ट्स करतात. अंतिम परिणाम दाखवण्यासाठी आम्ही `@executor` डेकोरेटर वापरून कस्टम एक्झिक्युटर तयार करतो.

**महत्त्वाचे संकल्पना:**
- `@executor(id="...")` - एखाद्या फंक्शनला वर्कफ्लो एक्झिक्युटर म्हणून नोंदवते
- `WorkflowContext[Never, str]` - इनपुट/आउटपुटसाठी टाइप हिंट्स
- `ctx.yield_output(...)` - अंतिम वर्कफ्लो परिणाम प्रदान करते


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## चरण ५: पर्यावरणीय व्हेरिएबल्स लोड करा

LLM क्लायंट कॉन्फिगर करा. हा उदाहरण खालीलसाठी कार्य करतो:
- **GitHub Models** (GitHub टोकनसह मोफत स्तर)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## चरण 6: संरचित आउटपुटसह AI एजंट तयार करा

आम्ही **तीन विशेष एजंट** तयार करतो, प्रत्येकाला `AgentExecutor` मध्ये गुंडाळले जाते:

1. **availability_agent** - साधन वापरून हॉटेलची उपलब्धता तपासतो
2. **alternative_agent** - पर्यायी शहरांची शिफारस करतो (जेव्हा खोल्या उपलब्ध नसतात)
3. **booking_agent** - बुकिंगसाठी प्रोत्साहन देतो (जेव्हा खोल्या उपलब्ध असतात)

**महत्त्वाच्या वैशिष्ट्ये:**
- `tools=[hotel_booking]` - एजंटला साधन उपलब्ध करून देते
- `response_format=PydanticModel` - संरचित JSON आउटपुट अनिवार्य करते
- `AgentExecutor(..., id="...")` - कार्यप्रवाहासाठी एजंटला गुंडाळते


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## चरण 7: अटींनुसार मार्गक्रमणासह कार्यप्रवाह तयार करा

आता आपण `WorkflowBuilder` वापरून अटींनुसार मार्गक्रमणासह ग्राफ तयार करतो:

**कार्यप्रवाह संरचना:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**महत्त्वाच्या पद्धती:**
- `.set_start_executor(...)` - प्रारंभ बिंदू सेट करते
- `.add_edge(from, to, condition=...)` - अटींनुसार मार्ग जोडते
- `.build()` - कार्यप्रवाह अंतिम रूप देते


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## चरण 8: चाचणी प्रकरण 1 चालवा - उपलब्धता नसलेले शहर (पॅरिस)

चला **उपलब्धता नसलेला** मार्ग तपासूया, पॅरिसमधील हॉटेल्सची विनंती करून (ज्यांच्याकडे आपल्या सिम्युलेशनमध्ये कोणतीही खोली नाही).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## चरण 9: चाचणी प्रकरण 2 चालवा - उपलब्धता असलेले शहर (स्टॉकहोम)

आता स्टॉकहोममधील हॉटेल्सची विनंती करून **उपलब्धता** मार्गाची चाचणी करूया (ज्यामध्ये आमच्या सिम्युलेशनमध्ये खोल्या आहेत).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## मुख्य मुद्दे आणि पुढील पावले

### ✅ तुम्ही काय शिकलात:

1. **WorkflowBuilder पॅटर्न**
   - प्रवेश बिंदू परिभाषित करण्यासाठी `.set_start_executor()` वापरा
   - सशर्त रूटिंगसाठी `.add_edge(from, to, condition=...)` वापरा
   - वर्कफ्लो अंतिम रूप देण्यासाठी `.build()` कॉल करा

2. **सशर्त रूटिंग**
   - Condition फंक्शन्स `AgentExecutorResponse` तपासतात
   - रूटिंग निर्णय घेण्यासाठी संरचित आउटपुट्स पार्स करा
   - एखाद्या एज सक्रिय करण्यासाठी `True` परत करा, ते टाळण्यासाठी `False` परत करा

3. **टूल इंटिग्रेशन**
   - Python फंक्शन्स AI टूल्समध्ये रूपांतरित करण्यासाठी `@ai_function` वापरा
   - एजंट्स आवश्यकतेनुसार टूल्स आपोआप कॉल करतात
   - टूल्स JSON परत करतात ज्याचा एजंट्स पार्स करू शकतात

4. **संरचित आउटपुट्स**
   - प्रकार-सुरक्षित डेटा एक्स्ट्रॅक्शनसाठी Pydantic मॉडेल्स वापरा
   - एजंट्स तयार करताना `response_format=MyModel` सेट करा
   - `Model.model_validate_json()` वापरून प्रतिसाद पार्स करा

5. **कस्टम Executors**
   - वर्कफ्लो घटक तयार करण्यासाठी `@executor(id="...")` वापरा
   - Executors डेटा रूपांतरित करू शकतात किंवा साइड इफेक्ट्स करू शकतात
   - वर्कफ्लो परिणाम तयार करण्यासाठी `ctx.yield_output()` वापरा

### 🚀 वास्तविक-जगातील उपयोग:

- **प्रवास बुकिंग**: उपलब्धता तपासा, पर्याय सुचवा, तुलना करा
- **ग्राहक सेवा**: समस्या प्रकार, भावना, प्राधान्य यावर आधारित रूटिंग
- **ई-कॉमर्स**: इन्व्हेंटरी तपासा, पर्याय सुचवा, ऑर्डर प्रक्रिया करा
- **कंटेंट मॉडरेशन**: विषारी स्कोर्स, वापरकर्ता फ्लॅग्सवर आधारित रूटिंग
- **मंजुरी वर्कफ्लो**: रक्कम, वापरकर्ता भूमिका, जोखीम स्तरावर आधारित रूटिंग
- **मल्टी-स्टेज प्रक्रिया**: डेटा गुणवत्ता, पूर्णतेवर आधारित रूटिंग

### 📚 पुढील पावले:

- अधिक जटिल अटी जोडा (एकाधिक निकष)
- वर्कफ्लो स्टेट मॅनेजमेंटसह लूप्स अंमलात आणा
- पुनर्वापरयोग्य घटकांसाठी उप-वर्कफ्लो जोडा
- वास्तविक APIs (हॉटेल बुकिंग, इन्व्हेंटरी सिस्टम्स) सह एकत्रित करा
- त्रुटी हाताळणी आणि फॉलबॅक पथ जोडा
- अंगभूत व्हिज्युअलायझेशन टूल्ससह वर्कफ्लो व्हिज्युअल करा



---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी कृपया लक्षात ठेवा की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर करून निर्माण होणाऱ्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
